In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("/content/drive/Shareddrives/DAB 경진대회/모델링/data/train_data.csv")
test = pd.read_csv("/content/drive/Shareddrives/DAB 경진대회/모델링/data/test_data.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train.head()

,체결 날짜,거래 사이즈,거래 가격,신발이름,브랜드,거래량,스크랩수,좋아요수,출시일,컬러,발매가
0,22/04/04,230,300000,Nike Dunk Low Retro Black,Nike,188000,101000,11000,2021-01-04,WHITE/BLACK,129000
1,22/06/29,240(US 5.5),250000,Nike Dunk Low Retro Black,Nike,188000,101000,11000,2021-01-04,WHITE/BLACK,129000
2,22/06/30,255,191000,Nike Dunk Low Retro Black,Nike,188000,101000,11000,2021-01-04,WHITE/BLACK,129000
3,22/06/30,255,194000,Nike Dunk Low Retro Black,Nike,188000,101000,11000,2021-01-04,WHITE/BLACK,129000
4,22/06/30,255,194000,Nike Dunk Low Retro Black,Nike,188000,101000,11000,2021-01-04,WHITE/BLACK,129000


## 전처리

1. 필요한 데이터만 추리기
2. 체결 날짜 - 출시일 (일 수로 변환)
3. 거래 가격 - 발매가 (얘는 잘 모르겠네)



In [ ]:
train = train[['체결 날짜', '거래 사이즈', '거래 가격', '출시일', '발매가']]
test = test[['체결 날짜', '거래 사이즈', '거래 가격', '출시일', '발매가']]

In [ ]:
from datetime import datetime
import re

In [ ]:
deal_date = []
launch_date = []
differ_date = []
for i in range(len(train)):
  a = datetime.strptime("20" + train['체결 날짜'][i].replace("/", ""), "%Y%m%d")
  b = datetime.strptime(train['출시일'][i].replace("-", ""), "%Y%m%d")
  deal_date.append(a)
  launch_date.append(b)
  differ_date.append((a-b).days)

train['체결 날짜'] = deal_date
train['출시일'] = launch_date
train['체결-출시'] = differ_date

In [ ]:
type(train['체결 날짜'][0]), type(train['출시일'][0]), type(train['체결-출시'][0]), train['체결-출시'][0]

(pandas._libs.tslibs.timestamps.Timestamp,
 pandas._libs.tslibs.timestamps.Timestamp,
 numpy.int64,
 455)

In [ ]:
deal_date = []
launch_date = []
differ_date = []
for i in range(len(test)):
  a = datetime.strptime("20" + test['체결 날짜'][i].replace("/", ""), "%Y%m%d")
  b = datetime.strptime(test['출시일'][i].replace("-", ""), "%Y%m%d")
  deal_date.append(a)
  launch_date.append(b)
  differ_date.append((a-b).days)

test['체결 날짜'] = deal_date
test['출시일'] = launch_date
test['체결-출시'] = differ_date

## ML 모델

In [ ]:
train.head()

,체결 날짜,거래 사이즈,거래 가격,출시일,발매가,체결-출시
0,2022-04-04,230,300000,2021-01-04,129000,455
1,2022-06-29,240(US 5.5),250000,2021-01-04,129000,541
2,2022-06-30,255,191000,2021-01-04,129000,542
3,2022-06-30,255,194000,2021-01-04,129000,542
4,2022-06-30,255,194000,2021-01-04,129000,542


In [ ]:
test.head()

,체결 날짜,거래 사이즈,거래 가격,출시일,발매가,체결-출시
0,2022-07-02,255,194000,2021-01-04,129000,544
1,2022-07-02,255,193000,2021-01-04,129000,544
2,2022-07-02,255,191000,2021-01-04,129000,544
3,2022-07-02,255,194000,2021-01-04,129000,544
4,2022-07-02,255,194000,2021-01-04,129000,544


In [ ]:
PATH = '/content/drive/Shareddrives/DAB 경진대회/모델링/김퉤퉤'

In [ ]:
## automl부터
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
# 기존 h2o 삭제
!pip uninstall h2o
# h2o 설치
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 177.0 MB 42 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.2-py2.py3-none-any.whl size=177070945 sha256=8150bbadce53bee2acc1603288cf6b3a460440dfd2f52a64ea61500ee6dd0f77
  Stored in directory: /root/.cache/pip/wheels/a5/2b/ae/534595df09001e1573b1046be62a3f049507307340119db65c
Successfully built h2o


In [ ]:
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import time
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
%matplotlib inline
h2o.init()
h2o.no_progress()
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmprc1mzfdv
  JVM stdout: /tmp/tmprc1mzfdv/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmprc1mzfdv/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.2
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_unknownUser_1bshhm
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
# 세 달 기준 moving average
train['MA1'] = train['거래 가격'].rolling(3).mean()

In [ ]:
import plotly.express as px

fig = px.line(train, x = '체결 날짜', y = ['거래 가격', "MA1"], template = 'plotly_dark')
fig.show()

혹시나 했는데 규칙성이라곤 전혀 없죠?

In [ ]:
# 시계열을 위해 다시 작업...
train_seq = train.copy()

train_seq['M'] = [i.month for i in train_seq['체결 날짜']]
train_seq['D'] = [i.day for i in train_seq['체결 날짜']]
#train_seq['Y'] = [i.year for i in train_seq['체결 날짜']]
train_seq = train_seq.sort_values(['M','D'], ascending = True)
train_seq['Series'] = np.arange(1,len(train_seq)+1)

train_seq.drop(['체결 날짜', 'MA1'], axis =1, inplace =True)
train_seq.head(-5)

,거래 사이즈,거래 가격,출시일,발매가,체결-출시,M,D,Series
14736,320,360000,2021-01-04,129000,429,3,9,1
3220,300,235000,2021-01-04,129000,450,3,30,2
15987,290,226000,2021-01-04,129000,450,3,30,3
16104,305,233000,2021-01-04,129000,450,3,30,4
16802,245,308000,2021-01-04,129000,450,3,30,5
...,...,...,...,...,...,...,...,...
17537,285,223000,2021-01-04,129000,542,6,30,19750
17538,285,196000,2021-01-04,129000,542,6,30,19751
17539,285,202000,2021-01-04,129000,542,6,30,19752
17540,285,200000,2021-01-04,129000,542,6,30,19753


In [ ]:
test_seq = test.copy()

test_seq['M'] = [i.month for i in test_seq['체결 날짜']]
test_seq['D'] = [i.day for i in test_seq['체결 날짜']]
#test_seq['Y'] = [i.year for i in test_seq['체결 날짜']]
test_seq = test_seq.sort_values(['M','D'], ascending = True)
test_seq['Series'] = np.arange(1,len(test_seq)+1)

test_seq.drop(['체결 날짜'], axis =1, inplace =True)
test_seq.head()

,거래 사이즈,거래 가격,출시일,발매가,체결-출시,M,D,Series
20,255,170000,2021-01-04,129000,543,7,1,1
21,255,193000,2021-01-04,129000,543,7,1,2
22,255,188000,2021-01-04,129000,543,7,1,3
23,255,185000,2021-01-04,129000,543,7,1,4
24,255,194000,2021-01-04,129000,543,7,1,5


데이터 수정 후 작업 진행 예정 : 참고 링크 = https://towardsdatascience.com/time-series-forecasting-with-pycaret-regression-module-237b703a0c63

In [ ]:
train_seq.to_csv("train_seq.csv", index = False)
test_seq.to_csv("test_seq.csv", index = False)

In [ ]:
train_seq_h2o = h2o.upload_file("/content/train_seq.csv")
test_seq_h2o = h2o.upload_file("/content/test_seq.csv")

In [ ]:
train_seq_h2o.head()

거래 사이즈,거래 가격,출시일,발매가,체결-출시,M,D,Series
320,360000,2021-01-04 00:00:00,129000,429,3,9,1
300,235000,2021-01-04 00:00:00,129000,450,3,30,2
290,226000,2021-01-04 00:00:00,129000,450,3,30,3
305,233000,2021-01-04 00:00:00,129000,450,3,30,4
245,308000,2021-01-04 00:00:00,129000,450,3,30,5
285,221000,2021-01-04 00:00:00,129000,450,3,30,6
300,257000,2021-01-04 00:00:00,129000,451,3,31,7
300,244000,2021-01-04 00:00:00,129000,451,3,31,8
300,240000,2021-01-04 00:00:00,129000,451,3,31,9
250,238000,2021-01-04 00:00:00,129000,451,3,31,10


In [ ]:
train_seq_h2o.columns[:2]

['거래 사이즈', '거래 가격']

In [ ]:
predictors = ['거래 사이즈', '거래 가격', '체결-출시', 'M', 'D']
train_seq_h2o['거래 가격'].isnumeric()
response = "거래 가격"

In [ ]:
train_h2o, test_h2o = train_seq_h2o.split_frame(ratios = [0.8], seed = 1234)

jordan_glm = H2OGeneralizedLinearEstimator(alpha = 0.25)
jordan_glm.train(x = predictors, y = response, training_frame = train_h2o)

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1657166211155_1


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,gaussian,identity,"Elastic Net (alpha = 0.25, lambda = 68.462 )",4,4,1,py_13_sid_82bc




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 577564990.4058211
RMSE: 24032.581850600676
MAE: 17565.463278333507
RMSLE: 0.1063586069744245
R^2: 0.0360690810321348
Mean Residual Deviance: 577564990.4058211
Null degrees of freedom: 15893
Residual degrees of freedom: 15889
Null deviance: 9523315184597.607
Residual deviance: 9179817957510.121
AIC: 365768.04345561

Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_deviance,training_mae,training_r2
0,,2022-07-07 04:25:58,0.000 sec,0,9523315184597.730469,599176744.972803,,,,
1,,2022-07-07 04:25:58,0.159 sec,1,,,24032.581851,577564990.405821,17565.463278,0.036069



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,체결-출시,320.900482,1.000000,0.406464
1,M,297.801422,0.928018,0.377206
2,D,93.040588,0.289936,0.117848
3,거래 사이즈,77.750725,0.242289,0.098482


In [ ]:
predict = jordan_glm.predict(test_h2o)
predict.head()

predict
209928
210198
210198
210198
210198
210198
210198
210198
210198
210198


In [ ]:
test_h2o['거래 가격']

거래 가격
233000
238000
245000
253000
253000
253000
244000
252000
237000
253000
